### Importing the Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
import tensorflow as tf

### Preprocessing the Training Data Set

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.image_dataset import image_dataset_from_directory
# train_datagen = ImageDataGenerator(
#     rescale = 1./255,
#     zoom_range = 0.2,
#     shear_range = 0.2,
#     horizontal_flip = True
# )

# trainingDataset = train_datagen.flow_from_directory(
#     '../Datasets/COVID-19_Chest_CT-Scan_Dataset/Training_Set/',
#     target_size = (224, 224),
#     batch_size = 32,
#     class_mode = 'categorical'
# )

trainingDataset = image_dataset_from_directory(
    '../Datasets/COVID-19_Chest_CT-Scan_Dataset/Training_Set/',
    labels = 'inferred',
    label_mode = 'categorical',
    image_size = (224, 224),
    color_mode = 'rgb'
)

print(trainingDataset)
print(trainingDataset.class_names)

Found 2000 files belonging to 2 classes.
<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>
['COVID', 'non-COVID']


### Preprocessing the Test Data Set

In [3]:
# test_datagen = ImageDataGenerator(
#     rescale = 1./255,
#     zoom_range = 0.2,
#     shear_range = 0.2,
#     horizontal_flip = True
# )

# testDataset = test_datagen.flow_from_directory(
#     '../Datasets/COVID-19_Chest_CT-Scan_Dataset/Test_Set/',
#     target_size = (224, 224),
#     batch_size = 32,
#     class_mode = 'categorical'
# )

testDataset = image_dataset_from_directory(
    '../Datasets/COVID-19_Chest_CT-Scan_Dataset/Test_Set/',
    labels = 'inferred',
    label_mode = 'categorical',
    image_size = (224, 224),
    color_mode = 'rgb'
)

print(testDataset.class_names)

Found 481 files belonging to 2 classes.
['COVID', 'non-COVID']


### Implementing the DenseNet-121 Architecture

In [4]:
from keras.layers import Flatten, Dense, AveragePooling2D, Dropout
from keras.applications import DenseNet121
from keras import Input
from keras.models import Model

denseNet = DenseNet121(include_top = False, weights = 'imagenet', input_tensor = Input(shape = (224, 224, 3)))
denseModel = denseNet.output
denseModel = AveragePooling2D(pool_size = (4, 4))(denseModel)
denseModel = Flatten()(denseModel)
denseModel = Dense(512, activation = "relu")(denseModel)
denseModel = Dropout(0.25)(denseModel)
denseModel = Dense(2, activation = "softmax")(denseModel)

for layer in denseNet.layers:
    layer.trainable = False

### Creating the Final Model

In [5]:
model = Model(inputs = denseNet.input, outputs = denseModel)
model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.001), loss = keras.losses.CategoricalCrossentropy(), metrics = [keras.metrics.CategoricalAccuracy(), keras.metrics.AUC(), keras.metrics.Accuracy()])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

In [6]:
finalModel = model.fit(x = trainingDataset, validation_data = testDataset, epochs = 10, verbose = 1)

Epoch 1/10
63/63 [==============================] - 449s 7s/step - loss: 1.7974 - categorical_accuracy: 0.6870 - auc: 0.7240 - accuracy: 0.0262 - val_loss: 1.5183 - val_categorical_accuracy: 0.6320 - val_auc: 0.6893 - val_accuracy: 0.0000e+00
Epoch 2/10
63/63 [==============================] - 390s 6s/step - loss: 0.5672 - categorical_accuracy: 0.7945 - auc: 0.8723 - accuracy: 0.0033 - val_loss: 0.9483 - val_categorical_accuracy: 0.6279 - val_auc: 0.6887 - val_accuracy: 0.0000e+00
Epoch 3/10
63/63 [==============================] - 391s 6s/step - loss: 0.4105 - categorical_accuracy: 0.8250 - auc: 0.9060 - accuracy: 0.0018 - val_loss: 0.9155 - val_categorical_accuracy: 0.6299 - val_auc: 0.6997 - val_accuracy: 0.0000e+00
Epoch 4/10
63/63 [==============================] - 385s 6s/step - loss: 0.3241 - categorical_accuracy: 0.8635 - auc: 0.9357 - accuracy: 2.5000e-04 - val_loss: 1.2165 - val_categorical_accuracy: 0.6299 - val_auc: 0.6768 - val_accuracy: 0.0000e+00
Epoch 5/10
63/63 [======

In [10]:
loss = list(model.evaluate(x = trainingDataset, batch_size = 32, verbose = 1))
print(*loss)
# print("Loss: {:.2f} AUC: {:.2f} Accuracy: {:.2f}".format(loss, auc, accuracy))

63/63 [==============================] - 314s 5s/step - loss: 0.1513 - categorical_accuracy: 0.9485 - auc: 0.9892 - accuracy: 0.0022
0.15130844712257385 0.9484999775886536 0.9891579151153564 0.0022499999031424522


In [18]:
from keras.preprocessing import image

testImage = image.image_utils.load_img('../Datasets/COVID-19_Chest_CT-Scan_Dataset/COVID/Covid (1006).png', target_size = (224, 224))
testImage = image.image_utils.img_to_array(testImage)
testImage = np.expand_dims(testImage, axis = 0)
predictionResult = model.predict(x = testImage, batch_size = 32)
print(trainingDataset.class_names)
print(predictionResult)

1/1 [==============================] - 0s 250ms/step
['COVID', 'non-COVID']
[[9.9999905e-01 9.5097960e-07]]
